In [1]:
#importing libraries 

#data manipulation
import pandas as pd 
import numpy as np

#display images
from IPython.display import Image

#databasing
import sqlite3
from sqlite3 import Error
from pathlib import Path

#API
import requests

### Information to be captured

##### Investor 1
For Investor 1, borrower information that will be captured through pinging the CreditScale API for borrower credit information (https://documenter.getpostman.com/view/3379028/S17xr5ZJ#d4b992dc-defe-46b8-adf1-eaea533318a9) will include:

1. Full payment history on all existing tradelines, both open and closed, in order to check that each tradeline is considered 'satisfactory' per prompt bullet #1
1. Full payment history on all mortgage lines and other installment loans in order to check that each mortgage line is considered 'satisfactory' per prompt bullet #2
1. Full payment history on all rental payments in order to check that rental payment history is considered 'satisfactory' per prompt bullet #3
1. Most recent applicant FICO score from Experian, Transunion and Equifax per prompt bullet #4 
1. Date of oldest existing liability in the applicant's credit history file, either open or closed, per prompt bullet #4
1. Information on when each tradeline was opened in order to satisfy tradeline minimum history requirements outlined in bullets #1, #2, #3 & #5
1. Information on whether each tradeline is currently open and active per prompt bullet #6

*Note*: As Verification of Rent requires a signed VOR Form from the landlord, this information will not be captured through these API requests 

*Note:* As proofs of cancelled checks require delivery of canceled checks, this information will not be captured through these API requests 



##### Investor 2
For Investor 2, borrower information that will be captured through pinging the CreditScale API will include:

1. Full payment history on all existing **active** tradelines in order to check that each tradeline is considered 'satisfactory' per prompt bullet #1
1. Information on when each active tradeline was opened in order to satisfy tradeline minimum history requirements outlined in bullet #1
1. Date of oldest existing liability in the applicant's credit history file in order to satisfy bullets #2 & #3
1. Most recent applicant FICO score from Experian, Transunion and Equifax in order to satisfy bullet #4
1. Full payment and activity history on all tradelines listed in credit file in order to satisfy bullets #5, #6, #7 & #8

### How information will be queried & stored 

#### How information will be queried

We will gather the above investor requirement information  through a series of GET requests to the CreditScale API on available tradeline, FICO score and credit file information for a unique applicant_unique_id per the below provided command line API calls and example JSON responses for each information subset.

CreditScale API documentation: https://documenter.getpostman.com/view/3379028/S17xr5ZJ#d4b992dc-defe-46b8-adf1-eaea533318a9

#### How information will be stored - Two approaches using mySQL & sqllite3 

We propose two different approaches to storing applicant data depending on the business's existing database practices and internal requirements for database table partitioning using mySQL and sqllite3. 

1. In the first approach using mySQL, we will create a single dataframe combining the above information for each tradeline for all applicant_unique_id's and export this to a csv file that will be imported through a local infile load operation into a pre-defined table schema in our mySQL environment leveraging (applicant_unique_id, tradeline_id) as a composite primary key to ensure table accessibility & integrity. The loading of subsequent tables following the same schema will then be automated using the bash script provided further below in this notebook. 

1. In the second approach using sqllite, we will create *individual* sqllite tables for each borrower applicant that captures the above listed information for investors 1 and 2. These individual tables will then be loaded using a function script into our existing sqllite database created for this exercise. 

Additional considerations for edge cases our implementation should be able to handle throughout these two implementations are provided at the end of this notebook as well.


### Credit Definitions:

In order to specify what constitues a 'satisfactory' tradeline per these investor requirements, the below definitions per outside research are used in this exercise: 

**Satisfactory Installment Loan Credit** = Made all housing and installment debt payments on time for the previous 12 months and have no more than two 30-day late mortgage or installment payments in the previous 24 months

**Satisfactory Revolving Loan Credit** = No major derogatory credit on revolving accounts in the previous 12 months. A revolving account is not considered derogatory as long as there are no 90-day late payments and no more than three 60-day late payments in the last 12 months.


Source: https://myhecm.com/reverse-mortgage-glossary/financial-assessment/financial-willingness/satisfactory-credit#:~:text=The%20satisfactory%20credit%20test%20helps,charges%20under%20the%20HECM%20program.

In [21]:
#defining a sample order_id for our API requests

order_id1 = 10001
order_id2 = 10002

### 1. CreditScale Tradeline API Request:

#### Request:

curl --location -g --request GET 'sandbox-api.creditscale.io/v1/credit/{{order_id}}/liabilities' \
--header 'Authorization: Basic {{api_key}}'

where order_id = 10001

#### Response:

In [17]:
#sample CreditScale Tradeline API responses #1
response_tradeline1 = {
  "liabilities": [
    {
      "sequence_number": 1,
      "id": "CreditLiability_1_1",
      "30_days_late_count": 1,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "925786821",
      "account_opened_date": "2017-01-01",
      "account_ownership_type": "JointContractualLiability",
      "account_paid_date": "",
      "account_reported_date": "2018-03-01",
      "account_status_type": "Open",
      "account_type": "Mortgage",
      "credit_business_type": "Finance",
      "credit_limit_amount": "",
      "credit_loan_type": "Mortgage",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "COUNTRYWIDE HOME LOANS",
      "highest_adverse_rating_code": "1",
      "highest_adverse_rating_date": "2017-12-01",
      "highest_adverse_rating_type": "Late30Days",
      "last_activity_date": "2018-03-01",
      "monthly_payment_amount": "1444.00",
      "most_recent_adverse_rating_code": "1",
      "most_recent_adverse_rating_date": "2018-03-01",
      "most_recent_adverse_rating_type": "Late30Days",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCC1CCCCCCC",
      "payment_pattern_start_date": "2018-03-01",
      "prior_adverse_rating_code": "1",
      "prior_adverse_rating_date": "2018-03-01",
      "prior_adverse_rating_type": "Late30Days",
      "terms_months_count": 360,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "118610.00"
    },
    {
      "sequence_number": 2,
      "id": "CreditLiability_1_2",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "6817741437",
      "account_opened_date": "2017-11-01",
      "account_ownership_type": "JointContractualLiability",
      "account_paid_date": "",
      "account_reported_date": "2018-03-01",
      "account_status_type": "Open",
      "account_type": "Mortgage",
      "credit_business_type": "Finance",
      "credit_limit_amount": "",
      "credit_loan_type": "Mortgage",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "GMAC MORTGAGE",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-03-01",
      "monthly_payment_amount": "201.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCC",
      "payment_pattern_start_date": "2018-03-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 300,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "24910.00"
    },
    {
      "sequence_number": 3,
      "id": "CreditLiability_1_3",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "77745013",
      "account_opened_date": "2016-01-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Installment",
      "credit_business_type": "Finance",
      "credit_limit_amount": "",
      "credit_loan_type": "Automobile",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "A H F C",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-03-01",
      "monthly_payment_amount": "247.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 60,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "8205.00"
    },
    {
      "sequence_number": 4,
      "id": "CreditLiability_1_4",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "5418221673200088",
      "account_opened_date": "2017-11-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "8000.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "WASH MUTUAL/PROVIDIAN",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-03-01",
      "monthly_payment_amount": "152.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "7568.00"
    },
    {
      "sequence_number": 5,
      "id": "CreditLiability_1_5",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "5149110921965684",
      "account_opened_date": "2015-05-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "5600.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "CHASE",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-04-01",
      "monthly_payment_amount": "107.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "5388.00"
    },
    {
      "sequence_number": 6,
      "id": "CreditLiability_1_6",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "517858968670",
      "account_opened_date": "2014-05-01",
      "account_ownership_type": "JointContractualLiability",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "CAP ONE BK",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-04-01",
      "monthly_payment_amount": "82.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "2746.00"
    },
    {
      "sequence_number": 7,
      "id": "CreditLiability_1_7",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "5166",
      "account_opened_date": "2014-09-01",
      "account_ownership_type": "AuthorizedUser",
      "account_paid_date": "",
      "account_reported_date": "2017-12-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "5200.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "BANK OF AMERICA",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-05-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2017-12-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 8,
      "id": "CreditLiability_1_8",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "6153",
      "account_opened_date": "2014-11-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "6800.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "BANK OF AMERICA",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-01-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 9,
      "id": "CreditLiability_1_9",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "601935133357",
      "account_opened_date": "2015-10-01",
      "account_ownership_type": "JointContractualLiability",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "5000.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "GEMB/CARE CREDIT CORE",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-04-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 10,
      "id": "CreditLiability_1_10",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "514046736631",
      "account_opened_date": "2017-07-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "1500.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "BARCLAYS BANK DELAWARE",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-11-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 11,
      "id": "CreditLiability_1_11",
      "30_days_late_count": 1,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2017-12-01",
      "account_identifier": "57404736",
      "account_opened_date": "2014-09-01",
      "account_ownership_type": "JointContractualLiability",
      "account_paid_date": "2018-01-01",
      "account_reported_date": "2018-01-01",
      "account_status_type": "Paid",
      "account_type": "Installment",
      "credit_business_type": "Finance",
      "credit_limit_amount": "",
      "credit_loan_type": "Automobile",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "FORD CRED",
      "highest_adverse_rating_code": "1",
      "highest_adverse_rating_date": "2017-09-01",
      "highest_adverse_rating_type": "Late30Days",
      "last_activity_date": "2018-01-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "1",
      "most_recent_adverse_rating_date": "2018-01-01",
      "most_recent_adverse_rating_type": "Late30Days",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "XCCC1CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-01-01",
      "prior_adverse_rating_code": "1",
      "prior_adverse_rating_date": "2018-01-01",
      "prior_adverse_rating_type": "Late30Days",
      "terms_months_count": 38,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 12,
      "id": "CreditLiability_1_12",
      "30_days_late_count": 1,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "540801007152",
      "account_opened_date": "2016-06-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-03-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "500.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "HSBC NV",
      "highest_adverse_rating_code": "1",
      "highest_adverse_rating_date": "2017-01-01",
      "highest_adverse_rating_type": "Late30Days",
      "last_activity_date": "2017-08-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "1",
      "most_recent_adverse_rating_date": "2018-03-01",
      "most_recent_adverse_rating_type": "Late30Days",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCC1CCCCCCC",
      "payment_pattern_start_date": "2018-03-01",
      "prior_adverse_rating_code": "1",
      "prior_adverse_rating_date": "2018-03-01",
      "prior_adverse_rating_type": "Late30Days",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 13,
      "id": "CreditLiability_1_13",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2012-02-01",
      "account_identifier": "45003879",
      "account_opened_date": "2007-06-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "2012-02-01",
      "account_reported_date": "2012-03-01",
      "account_status_type": "Paid",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "200.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "VNB/CREDIT LINES",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2012-02-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "XCCXXXXXXXXXXXXXXXXXXXXXX",
      "payment_pattern_start_date": "2012-03-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 14,
      "id": "CreditLiability_1_14",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "00403655342746",
      "account_opened_date": "2000-08-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2013-02-01",
      "account_status_type": "Paid",
      "account_type": "Open",
      "credit_business_type": "Banking",
      "credit_limit_amount": "",
      "credit_loan_type": "UnknownLoanType",
      "credit_repository_source_type": "Experian",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "AMEX",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "X",
      "payment_pattern_start_date": "2013-02-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 1,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 15,
      "id": "CreditLiability_1_15",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "00403655342742",
      "account_opened_date": "2001-02-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2013-02-01",
      "account_status_type": "Paid",
      "account_type": "Open",
      "credit_business_type": "Banking",
      "credit_limit_amount": "",
      "credit_loan_type": "UnknownLoanType",
      "credit_repository_source_type": "Experian",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "AMEX",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "X",
      "payment_pattern_start_date": "2013-02-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 1,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 16,
      "id": "CreditLiability_1_16",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2014-11-01",
      "account_identifier": "1640762743",
      "account_opened_date": "2011-12-01",
      "account_ownership_type": "Comaker",
      "account_paid_date": "2014-11-01",
      "account_reported_date": "2015-01-01",
      "account_status_type": "Paid",
      "account_type": "Installment",
      "credit_business_type": "Finance",
      "credit_limit_amount": "",
      "credit_loan_type": "Automobile",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "TOYOTA MOTOR CREDIT CO",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2014-11-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "XCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2015-01-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 36,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 17,
      "id": "CreditLiability_1_17",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2016-11-01",
      "account_identifier": "-099663074258484517",
      "account_opened_date": "2000-02-01",
      "account_ownership_type": "Terminated",
      "account_paid_date": "2017-02-01",
      "account_reported_date": "2017-03-01",
      "account_status_type": "Paid",
      "account_type": "Open",
      "credit_business_type": "Banking",
      "credit_limit_amount": "",
      "credit_loan_type": "UnknownLoanType",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "AMEX",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-02-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "X",
      "payment_pattern_start_date": "2017-03-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 1,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 18,
      "id": "CreditLiability_1_18",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2017-04-01",
      "account_identifier": "5459619610",
      "account_opened_date": "2015-03-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "2017-03-01",
      "account_reported_date": "2017-04-01",
      "account_status_type": "Paid",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "3500.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "WASH MUTUAL/PROVIDIAN",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-03-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "XCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2017-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 19,
      "id": "CreditLiability_1_19",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2009-12-01",
      "account_identifier": "411300001250",
      "account_opened_date": "1999-03-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "2009-12-01",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Paid",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "1700.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Experian",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "VALLEY NATIONAL BANK M",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2009-12-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "XCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 20,
      "id": "CreditLiability_1_20",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "431914269152",
      "account_opened_date": "2014-11-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "6800.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Equifax",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "BK OF AMER",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-04-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "C",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 21,
      "id": "CreditLiability_1_21",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "430597099545",
      "account_opened_date": "2014-09-01",
      "account_ownership_type": "AuthorizedUser",
      "account_paid_date": "",
      "account_reported_date": "2017-12-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "5200.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Equifax",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "BK OF AMER",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-05-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "",
      "payment_pattern_start_date": "",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    }
  ]
}


In [16]:
#sample CreditScale Tradeline API response #2
response_tradeline2 = {
  "liabilities": [
    {
      "sequence_number": 2,
      "id": "CreditLiability_1_2",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "6817741437",
      "account_opened_date": "2017-11-01",
      "account_ownership_type": "JointContractualLiability",
      "account_paid_date": "",
      "account_reported_date": "2018-03-01",
      "account_status_type": "Open",
      "account_type": "Mortgage",
      "credit_business_type": "Finance",
      "credit_limit_amount": "",
      "credit_loan_type": "Mortgage",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "GMAC MORTGAGE",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-03-01",
      "monthly_payment_amount": "201.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCC",
      "payment_pattern_start_date": "2018-03-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 300,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "24910.00"
    },
    {
      "sequence_number": 4,
      "id": "CreditLiability_1_4",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "5418221673200088",
      "account_opened_date": "2017-11-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "8000.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "WASH MUTUAL/PROVIDIAN",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-03-01",
      "monthly_payment_amount": "152.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "7568.00"
    },
    {
      "sequence_number": 5,
      "id": "CreditLiability_1_5",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "5149110921965684",
      "account_opened_date": "2015-05-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "5600.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "CHASE",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-04-01",
      "monthly_payment_amount": "107.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "5388.00"
    },
    {
      "sequence_number": 7,
      "id": "CreditLiability_1_7",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "5166",
      "account_opened_date": "2014-09-01",
      "account_ownership_type": "AuthorizedUser",
      "account_paid_date": "",
      "account_reported_date": "2017-12-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "5200.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "BANK OF AMERICA",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-05-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2017-12-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
  
    {
      "sequence_number": 14,
      "id": "CreditLiability_1_14",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "00403655342746",
      "account_opened_date": "2000-08-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2013-02-01",
      "account_status_type": "Paid",
      "account_type": "Open",
      "credit_business_type": "Banking",
      "credit_limit_amount": "",
      "credit_loan_type": "UnknownLoanType",
      "credit_repository_source_type": "Experian",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "AMEX",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "X",
      "payment_pattern_start_date": "2013-02-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 1,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 15,
      "id": "CreditLiability_1_15",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "00403655342742",
      "account_opened_date": "2001-02-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2013-02-01",
      "account_status_type": "Paid",
      "account_type": "Open",
      "credit_business_type": "Banking",
      "credit_limit_amount": "",
      "credit_loan_type": "UnknownLoanType",
      "credit_repository_source_type": "Experian",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "AMEX",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "X",
      "payment_pattern_start_date": "2013-02-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 1,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 16,
      "id": "CreditLiability_1_16",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2014-11-01",
      "account_identifier": "1640762743",
      "account_opened_date": "2011-12-01",
      "account_ownership_type": "Comaker",
      "account_paid_date": "2014-11-01",
      "account_reported_date": "2015-01-01",
      "account_status_type": "Paid",
      "account_type": "Installment",
      "credit_business_type": "Finance",
      "credit_limit_amount": "",
      "credit_loan_type": "Automobile",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "TOYOTA MOTOR CREDIT CO",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2014-11-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "XCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2015-01-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 36,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 17,
      "id": "CreditLiability_1_17",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2016-11-01",
      "account_identifier": "-099663074258484517",
      "account_opened_date": "2000-02-01",
      "account_ownership_type": "Terminated",
      "account_paid_date": "2017-02-01",
      "account_reported_date": "2017-03-01",
      "account_status_type": "Paid",
      "account_type": "Open",
      "credit_business_type": "Banking",
      "credit_limit_amount": "",
      "credit_loan_type": "UnknownLoanType",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "AMEX",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-02-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "X",
      "payment_pattern_start_date": "2017-03-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 1,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 18,
      "id": "CreditLiability_1_18",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2017-04-01",
      "account_identifier": "5459619610",
      "account_opened_date": "2015-03-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "2017-03-01",
      "account_reported_date": "2017-04-01",
      "account_status_type": "Paid",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "3500.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Other",
      "credit_repository_source_type_other_description": "MergedData",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "WASH MUTUAL/PROVIDIAN",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-03-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "XCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2017-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 19,
      "id": "CreditLiability_1_19",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "2009-12-01",
      "account_identifier": "411300001250",
      "account_opened_date": "1999-03-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "2009-12-01",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Paid",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "1700.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Experian",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "VALLEY NATIONAL BANK M",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2009-12-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "XCCCCCCCCCCCCCCCCCCCCCCCC",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 20,
      "id": "CreditLiability_1_20",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "431914269152",
      "account_opened_date": "2014-11-01",
      "account_ownership_type": "Individual",
      "account_paid_date": "",
      "account_reported_date": "2018-04-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "6800.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Equifax",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "BK OF AMER",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2018-04-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "C",
      "payment_pattern_start_date": "2018-04-01",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    },
    {
      "sequence_number": 21,
      "id": "CreditLiability_1_21",
      "30_days_late_count": 0,
      "60_days_late_count": 0,
      "90_days_late_count": 0,
      "account_closed_date": "",
      "account_identifier": "430597099545",
      "account_opened_date": "2014-09-01",
      "account_ownership_type": "AuthorizedUser",
      "account_paid_date": "",
      "account_reported_date": "2017-12-01",
      "account_status_type": "Open",
      "account_type": "Revolving",
      "credit_business_type": "Banking",
      "credit_limit_amount": "5200.00",
      "credit_loan_type": "CreditCard",
      "credit_repository_source_type": "Equifax",
      "credit_repository_source_type_other_description": "",
      "current_rating_code": "C",
      "current_rating_type": "AsAgreed",
      "full_name": "BK OF AMER",
      "highest_adverse_rating_code": "",
      "highest_adverse_rating_date": "",
      "highest_adverse_rating_type": "",
      "last_activity_date": "2017-05-01",
      "monthly_payment_amount": "0.00",
      "most_recent_adverse_rating_code": "",
      "most_recent_adverse_rating_date": "",
      "most_recent_adverse_rating_type": "",
      "past_due_amount": "0.00",
      "payment_pattern_data_text": "",
      "payment_pattern_start_date": "",
      "prior_adverse_rating_code": "",
      "prior_adverse_rating_date": "",
      "prior_adverse_rating_type": "",
      "terms_months_count": 0,
      "terms_source_type": "Provided",
      "unpaid_balance_amount": "0.00"
    }
  ]
}


### 2. CreditScale FICO Score API Request:

#### Request:

curl --location -g --request GET 'sandbox-api.creditscale.io/v1/credit/{{order_id}}/scores' \
--header 'Authorization: Basic {{your_api_key}}'

where order_id = 10001

#### Response:

In [23]:
#sample CreditScale FICO Score API response #1
response_fico1 = {
  "scores": [
    {
      "bureau": "Experian",
      "score": 697,
      "credit_score_model_name": "ExperianFairIsaac",
      "score_date": "2018-04-21"
    },
    {
      "bureau": "TransUnion",
      "score": 725,
      "credit_score_model_name": "FICORiskScoreClassic04",
      "score_date": "2018-04-21"
    },
    {
      "bureau": "Equifax",
      "score": 700,
      "credit_score_model_name": "EquifaxBeacon5.0",
      "score_date": "2018-04-21"
    }
  ]
}

In [19]:
#sample CreditScale FICO Score API response #2
response_fico2 = {
  "scores": [
    {
      "bureau": "Experian",
      "score": 569,
      "credit_score_model_name": "ExperianFairIsaac",
      "score_date": "2018-04-21"
    },
    {
      "bureau": "TransUnion",
      "score": 539,
      "credit_score_model_name": "FICORiskScoreClassic04",
      "score_date": "2018-04-21"
    },
    {
      "bureau": "Equifax",
      "score": 550,
      "credit_score_model_name": "EquifaxBeacon5.0",
      "score_date": "2018-04-21"
    }
  ]
}

### 3. CreditScale Credit File API Request:

#### Request:

curl --location -g --request GET 'sandbox-api.creditscale.io/v1/credit/{{order_id}}/summaries' \
--header 'Authorization: Basic {{your_api_key}}'

where order_id = 10001

#### Response:

In [25]:
#sample CreditScale Credit File API response #1
response_credit_summary1 = {
  "summaries": [
    {
      "trade_summary": {
        "auto_balance": "8205",
        "auto_count": "3",
        "auto_high_credit": "14162",
        "auto_past_due": "0",
        "auto_payment": "247",
        "education_balance": "0",
        "education_count": "0",
        "education_high_credit": "0",
        "education_past_due": "0",
        "education_payment": "0",
        "installment_balance": "8205",
        "installment_count": "3",
        "installment_high_credit": "14162",
        "installment_other_balance": "0",
        "installment_other_count": "0",
        "installment_other_high_credit": "0",
        "installment_other_past_due": "0",
        "installment_other_payment": "0",
        "installment_past_due": "0",
        "installment_payment": "247",
        "liability_oldest_creditor_name": "VALLEY NATIONAL BANK M",
        "liability_oldest_date": "199903",
        "mortgage_balance": "143520",
        "mortgage_count": "2",
        "mortgage_high_credit": "145000",
        "mortgage_past_due": "0",
        "mortgage_payment": "1645",
        "open_balance": "0",
        "open_count": "3",
        "open_high_credit": "0",
        "open_past_due": "0",
        "open_payment": "0",
        "revolving_balance": "15702",
        "revolving_count": "13",
        "revolving_high_credit": "47647",
        "revolving_past_due": "0",
        "revolving_payment": "341",
        "total_liability_balance": "167427",
        "total_liability_count": "21",
        "total_liability_high_credit": "206809",
        "total_liability_past_due": "0",
        "total_liability_payment": "2233",
        "unknown_balance": "0",
        "unknown_count": "0",
        "unknown_high_credit": "0",
        "unknown_past_due": "0",
        "unknown_payment": "0"
      },
      "derogatory_summary": {
        "total_secured_loan_balance": "151725",
        "total_unsecured_loan_balance": "15702",
        "total_high_credit": "206809",
        "debt_high_credit": "81",
        "revolving_credit_utilization": "33",
        "30_days_late_count": "3",
        "60_days_late_count": "0",
        "90_days_late_count": "0",
        "derog_other_count": "0",
        "liability_satisfactory_count": "18",
        "liability_bankruptcy_count": "0",
        "liability_collection_count": "0",
        "liability_charge_off_count": "0",
        "liability_current_adverse_count": "0",
        "liability_previous_adverse_count": "3",
        "inquiry_count": "26",
        "public_record_count": "0",
        "dispute_count": "0"
      }
    }
  ]
}

In [26]:
#sample CreditScale Credit File API response #2
response_credit_summary2 = {
  "summaries": [
    {
      "trade_summary": {
        "auto_balance": "9990",
        "auto_count": "3",
        "auto_high_credit": "14162",
        "auto_past_due": "0",
        "auto_payment": "247",
        "education_balance": "0",
        "education_count": "0",
        "education_high_credit": "0",
        "education_past_due": "0",
        "education_payment": "0",
        "installment_balance": "8205",
        "installment_count": "3",
        "installment_high_credit": "14162",
        "installment_other_balance": "0",
        "installment_other_count": "0",
        "installment_other_high_credit": "0",
        "installment_other_past_due": "0",
        "installment_other_payment": "0",
        "installment_past_due": "0",
        "installment_payment": "247",
        "liability_oldest_creditor_name": "VALLEY NATIONAL BANK M",
        "liability_oldest_date": "19903",
        "mortgage_balance": "14320",
        "mortgage_count": "2",
        "mortgage_high_credit": "145000",
        "mortgage_past_due": "0",
        "mortgage_payment": "1645",
        "open_balance": "0",
        "open_count": "3",
        "open_high_credit": "0",
        "open_past_due": "0",
        "open_payment": "0",
        "revolving_balance": "1572",
        "revolving_count": "13",
        "revolving_high_credit": "47647",
        "revolving_past_due": "0",
        "revolving_payment": "341",
        "total_liability_balance": "167427",
        "total_liability_count": "21",
        "total_liability_high_credit": "20609",
        "total_liability_past_due": "0",
        "total_liability_payment": "2233",
        "unknown_balance": "0",
        "unknown_count": "0",
        "unknown_high_credit": "0",
        "unknown_past_due": "0",
        "unknown_payment": "0"
      },
      "derogatory_summary": {
        "total_secured_loan_balance": "15125",
        "total_unsecured_loan_balance": "15702",
        "total_high_credit": "206809",
        "debt_high_credit": "81",
        "revolving_credit_utilization": "37",
        "30_days_late_count": "3",
        "60_days_late_count": "0",
        "90_days_late_count": "0",
        "derog_other_count": "0",
        "liability_satisfactory_count": "20",
        "liability_bankruptcy_count": "0",
        "liability_collection_count": "0",
        "liability_charge_off_count": "0",
        "liability_current_adverse_count": "0",
        "liability_previous_adverse_count": "3",
        "inquiry_count": "26",
        "public_record_count": "0",
        "dispute_count": "0"
      }
    }
  ]
}

In [ ]:
#creating function that returns dataframes containing relevant information on a borrower and which will be used
#to create our mySQL & sqllite tables

def create_applicant_table(applicant_unique_id = order_id1, response_fico = response_fico1,
                           response_tradeline = response_tradeline1, 
                           response_credit_summary = response_credit_summary1):
    
    '''
    function that takes in API's responses as inputs and outputs a dataframe containing information on applicant id's
    tradeline, credit score and credit summary histories per provided prompt investor data requirements 
    '''

    #instantiating tradeline history dataframe 
    tradeline_df = pd.DataFrame(columns = ['applicant_unique_id','tradeline_id','account_opened_date',
                                '30_days_late_count','60_days_late_count','90_days_late_count',
                                'credit_loan_type','account_type','payment_pattern_data_text',
                                'account_status_type','last_activity_date','highest_adverse_rating_date',
                                'highest_adverse_rating_type'])
    
    #instantiating fico score and other credit history dataframe
    fico_df = pd.DataFrame(columns = ['applicant_unique_id','Experian_score','TransUnion_score','Equifax_score',
                                     'Experian_score_date','TransUnion_score_date','Equifax_score_date'])
    
    #instantiating credit summary dataframe
    credit_summary_df = pd.DataFrame(columns = ['applicant_unique_id','liability_oldest_date'])
    
    #tradeline_df
    for k, v in response_tradeline.items():
        for index, dicts in enumerate(v):
            try:
                tradeline_df.loc[index] = [applicant_unique_id] + [dicts['id']] + [dicts['account_opened_date']] + \
                                [dicts['30_days_late_count']] + [dicts['60_days_late_count']] + \
                                [dicts['90_days_late_count']] + [dicts['credit_loan_type']] + \
                                [dicts['account_type']] + [dicts['payment_pattern_data_text']] + \
                                [dicts['account_status_type']] + [dicts['last_activity_date']] + \
                                [dicts['highest_adverse_rating_date']] + [dicts['highest_adverse_rating_type']] 
                                
            except:
                pass
    
    #fico_df
    for k, v in response_fico.items():
        for index, dicts in enumerate(v):
            if 'TransUnion' in dicts.values():
                transunion_score, transunion_score_date = dicts['score'], dicts['score_date']
            elif 'Experian' in dicts.values():
                experian_score, experian_score_date = dicts['score'], dicts['score_date']
            else:
                equifax_score, equifax_score_date = dicts['score'], dicts['score_date']
            try:
                fico_df.loc[index] = [applicant_unique_id] + [transunion_score] + [experian_score] + [equifax_score] + \
                                     [transunion_score_date] + [experian_score_date] + [equifax_score_date]

            except:
                pass
    
    #credit_summary_df
    for k, v in response_credit_summary.items():
        try:
            credit_summary_df.loc[index] = [applicant_unique_id] + [v[0]['trade_summary']['liability_oldest_date']]

        except:
            pass
            
    temp = pd.merge(tradeline_df, fico_df, how = 'left', on = 'applicant_unique_id')
    
    merged2 = pd.merge(temp, credit_summary_df, how = 'left', on = 'applicant_unique_id')
                
    return merged2

create_applicant_table(order_id1, response_fico1, response_tradeline1, response_credit_summary1)

In [28]:
#sample applicant API call dictionary 
applicant_10001_dict = {order_id1: [response_fico1, response_tradeline1, response_credit_summary1]}
applicant_10002_dict = {order_id2: [response_fico2, response_tradeline2, response_credit_summary2]}

### Approach 1: mySQL implementation

We would look to export to csv our dataframe containing all tradeline observations across all applicant_unique_id's and load this csv into a mySQL table setting (applicant_unique_id, tradeline_id) as a composite primary key to ensure table accessibility and integrity per the below mySQL script. The loading of subsequent tables following the same schema will be further automated using the bash script provided below. 


In [ ]:
#creating our csv files
df1 = create_applicant_table(order_id1, response_fico1, response_tradeline1, response_credit_summary1)
df1 = df1.append(create_applicant_table(order_id2, response_fico2, response_tradeline2, response_credit_summary2))
df1 = df1.reset_index(drop = True)
df1.to_csv('betterdf.csv', index = False)

#### mySQL Script

``` mysql
create database better;
USE better;
SET @@GLOBAL.local_infile = 1;
SET GLOBAL local_infile=1;
CREATE TABLE borrowers (
	applicant_unique_id INT,
    tradeline_id VARCHAR(255),
    account_opened_date	DATE DEFAULT NULL,
    days_late_count_30 INT DEFAULT NULL,
	days_late_count_60 INT DEFAULT NULL,
	days_late_count_90 INT DEFAULT NULL,
    credit_loan_type VARCHAR(255) DEFAULT NULL,	
    account_type VARCHAR(255) DEFAULT NULL,
    payment_pattern_data_text VARCHAR(255) DEFAULT NULL,
    account_status_type	VARCHAR(255) DEFAULT NULL,
    last_activity_date DATE DEFAULT NULL,
    highest_adverse_rating_date	DATE DEFAULT NULL,
    highest_adverse_rating_type	VARCHAR(255) DEFAULT NULL,
    Experian_score INT DEFAULT NULL,
    TransUnion_score INT DEFAULT NULL,
    Equifax_score INT DEFAULT NULL,
    Experian_score_date	DATE DEFAULT NULL,
    TransUnion_score_date DATE DEFAULT NULL,
    Equifax_score_date DATE DEFAULT NULL,
    liability_oldest_date INT DEFAULT NULL,
    CONSTRAINT Id_Tradeline_type PRIMARY KEY (applicant_unique_id, tradeline_id)
);

LOAD DATA LOCAL INFILE '/Users/philippeheitzmann/NYCDataScienceAcademy/Better.com_Case_Study_Marketplace_Analyst/betterdf.csv'
INTO TABLE borrowers
FIELDS TERMINATED BY ',' ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES
;
'''


We would then automate adding new tables to our database through using the following terminal commands:

In [ ]:
%db=better_db
%echo "creating database"
%mysql -e "DROP DATABASE IF EXISTS $db;CREATE DATABASE $db;"
%echo "loading tables"
%mysql $db < better.sql
%mysql -e "SHOW TABLES FROM $db;"

### Approach 2: sqllite implementation

We would look to automate adding *individual* tables for each applicant_unique_id to an sqllite database per the below sqllite database implementation and function script:


In [ ]:
#creating credit history SQL database 
database = 'credit_history.db'
Path(database).touch()
conn = sqlite3.connect(database)
c = conn.cursor()

In [47]:
def create_table(applicant_unique_id_dict = applicant_10001_dict):

    '''
    function that takes in a dict of applicant_unique_id keys referencing lists of credit bureau API responses as 
    inputs and adds to our existing sqllite database unique tables for each applicant_unique_id
    '''
    
    table_names = []
    if len(applicant_unique_id_dict) > 0:
        try:
            for k, v in applicant_unique_id_dict.items():
                df_name = 'applicant_' + str(k) + '_table'
                table_names.append(df_name)
                df = create_applicant_table(k, v[0], v[1], v[2])
                df.to_sql(df_name, conn, if_exists='replace', index = False)
        except:
            pass 
    else:
        return 'Please input a non-empty applicant dictionary'

    return 'Tables added successfully to existing database'#c.execute('SELECT * from {}').format(table_names[0]).fetchall()
    
create_table(applicant_unique_id_dict)

'Tables added successfully to existing database'

In [49]:
#testing our sqllite tables
c.execute('''SELECT * from applicant_10001_table''').fetchall()

[(10001,
  'CreditLiability_1_1',
  '2017-01-01',
  1,
  0,
  0,
  'Mortgage',
  'Mortgage',
  'CCC1CCCCCCC',
  'Open',
  '2018-03-01',
  '2017-12-01',
  'Late30Days',
  725,
  697,
  700,
  '2018-04-21',
  '2018-04-21',
  '2018-04-21',
  '199903'),
 (10001,
  'CreditLiability_1_2',
  '2017-11-01',
  0,
  0,
  0,
  'Mortgage',
  'Mortgage',
  'CCCCC',
  'Open',
  '2018-03-01',
  '',
  '',
  725,
  697,
  700,
  '2018-04-21',
  '2018-04-21',
  '2018-04-21',
  '199903'),
 (10001,
  'CreditLiability_1_3',
  '2016-01-01',
  0,
  0,
  0,
  'Automobile',
  'Installment',
  'CCCCCCCCCCCCCCCCCCCCCCCCC',
  'Open',
  '2018-03-01',
  '',
  '',
  725,
  697,
  700,
  '2018-04-21',
  '2018-04-21',
  '2018-04-21',
  '199903'),
 (10001,
  'CreditLiability_1_4',
  '2017-11-01',
  0,
  0,
  0,
  'CreditCard',
  'Revolving',
  'CCCCC',
  'Open',
  '2018-03-01',
  '',
  '',
  725,
  697,
  700,
  '2018-04-21',
  '2018-04-21',
  '2018-04-21',
  '199903'),
 (10001,
  'CreditLiability_1_5',
  '2015-05-01',


### Additional Considerations 
Additional considerations & edge cases that our system should be able to handle:

1. Record immediately preceding applicant_unique_id provided to the API at each API call to ensure that in the event that a daily or hourly API request limit is reached the business will be able to pick up at this last recorded API request and therefore ensure data completeness

1. Record the immediately preceding primary key stop-point after each API request to keep track of which tradeline observations have and have not been included for an individual applicant_unique_id in order to handle cases where the API would dynamically insert new tradeline rows into a credit file during the data collection process

1. Compare data counts and other aggregation checks on our loaded data in our SQL table vs our original csv file in order to ensure data completeness after export and local infile load 

1. Enforce locking of tables in our database in order to ensure data integrity and completeness & ensure that our database structure follows relational database requirements of atomicity, consistency, isolation and durability for best practices
